In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
import glob
import cv2
from tensorflow.keras.utils import img_to_array

2023-11-10 12:44:42.168882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Preprocessing

In [2]:
# pull images from david/ folder
images_dataset = []
labels_dataset = []
images_test_dataset = []
labels_test_dataset = []
img_rows, img_cols = 64, 64

for image_path in glob.glob("../../utils/cropped/2023-11-05-17-29-38/*.jpg"):
    image = cv2.imread(image_path)
    #print(image.shape)
    # need to bounding box here
    image = cv2.resize(image, (img_rows, img_cols))
    image = img_to_array(image)
    images_dataset.append(image)
    label = image_path.split("/")[-1].split("_")[0]
    labels_dataset.append(label)

for image_path in glob.glob("../../utils/cropped/2023-11-05-17-31-29/*.jpg"):
    image = cv2.imread(image_path)
    print(image.shape, image_path)
    # need to bounding box here
    image = cv2.resize(image, (img_rows, img_cols))
    image = img_to_array(image)
    images_test_dataset.append(image)
    label = "david" if "test_david" in image_path else "other"
    labels_test_dataset.append(label)

(228, 228, 3) ../../utils/cropped/2023-11-05-17-31-29/test_will_smith.jpg_2.jpg


In [3]:
# for each image in images_dataset, apply bounding box
for image in images_dataset:
    pass
    # apply bounding box
    

In [5]:
def create_model():
    inputs = Input((img_rows, img_cols, 3))
    x = Conv2D(96, (11, 11), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (5, 5), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(384, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    pooledOutput = GlobalAveragePooling2D()(x)
    pooledOutput = Dense(1024)(pooledOutput)
    outputs = Dense(128)(pooledOutput)

    model = Model(inputs, outputs)
    return model

In [6]:
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K

feature_extractor = create_model()
imgA = Input(shape=(img_rows, img_cols, 3))
imgB = Input(shape=(img_rows, img_cols, 3))
featA = feature_extractor(imgA)
featB = feature_extractor(imgB)

def euclidean_distance(vectors):
    (featA, featB) = vectors
    sum_squared = K.sum(K.square(featA - featB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

distance = Lambda(euclidean_distance)([featA, featB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

In [7]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "precision", "recall"])

In [7]:
import numpy as np

def generate_train_image_pairs(images_dataset, labels_dataset):
    unique_labels = np.unique(labels_dataset)
    print 
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels_dataset) if
                                       label == curr_label])
    
    pair_images = []
    pair_labels = []
    for index, image in enumerate(images_dataset):
        pos_indices = label_wise_indices.get(labels_dataset[index])
        pos_image = images_dataset[np.random.choice(pos_indices)]
        pair_images.append((image, pos_image))
        pair_labels.append(1)

        neg_indices = np.where(labels_dataset != labels_dataset[index])
        neg_image = images_dataset[np.random.choice(neg_indices[0])]
        pair_images.append((image, neg_image))
        pair_labels.append(0)
    return np.array(pair_images), np.array(pair_labels)

In [8]:
## Model training

images_pair, labels_pair = generate_train_image_pairs(images_dataset, labels_dataset)
history = model.fit([images_pair[:, 0], images_pair[:, 1]], labels_pair[:],validation_split=0.1,batch_size=4,epochs=5)

Epoch 1/5
5/5 [==============================] - 5s 632ms/step - loss: 19.6600 - accuracy: 0.4737 - val_loss: 0.8089 - val_accuracy: 0.6667
Epoch 2/5
5/5 [==============================] - 4s 742ms/step - loss: 1.7040 - accuracy: 0.4737 - val_loss: 0.5844 - val_accuracy: 0.6667
Epoch 3/5
5/5 [==============================] - 2s 477ms/step - loss: 0.7672 - accuracy: 0.4737 - val_loss: 0.6497 - val_accuracy: 0.6667
Epoch 4/5
5/5 [==============================] - 2s 422ms/step - loss: 0.7200 - accuracy: 0.4737 - val_loss: 0.6285 - val_accuracy: 0.6667
Epoch 5/5
5/5 [==============================] - 2s 388ms/step - loss: 0.7294 - accuracy: 0.4737 - val_loss: 0.6691 - val_accuracy: 0.6667


In [9]:
labels_pair

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

# TESTING

In [10]:
def generate_test_image_pairs(images_dataset, labels_dataset, image):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label, [index for index, curr_label in enumerate(labels_dataset) if label == curr_label])
  
    pair_images = []
    pair_labels = []
    for label, indices_for_label in label_wise_indices.items():
        test_image = images_dataset[np.random.choice(indices_for_label)]
        pair_images.append((image, test_image))
        pair_labels.append(label)
    return np.array(pair_images), np.array(pair_labels)

In [11]:
def predict(image):
  test_image_pairs, test_label_pairs = generate_test_image_pairs(images_dataset, labels_dataset, image) # produce an array of test image pairs and test label pairs
  #print("Test pairs", test_image_pairs)
  print("Test labels", test_label_pairs)
  for index, pair in enumerate(test_image_pairs):
      pair_image1 = np.expand_dims(pair[0], axis=-1)
      pair_image1 = np.expand_dims(pair_image1, axis=0)
      pair_image2 = np.expand_dims(pair[1], axis=-1)
      pair_image2 = np.expand_dims(pair_image2, axis=0)
      print(model.predict([pair_image1, pair_image2]))
      prediction = model.predict([pair_image1, pair_image2])[0][0]
      print("Test image pair {} similarity: {:.2f}".format(index, prediction))

In [12]:
print(labels_test_dataset)

# show the first image
predict(images_test_dataset[0])


['other']
Test labels ['david' 'will-smith']


1/1 [==============================] - 0s 272ms/step
[[0.46272734]]
1/1 [==============================] - 0s 65ms/step
Test image pair 0 similarity: 0.46
1/1 [==============================] - 0s 69ms/step
[[0.48013836]]
1/1 [==============================] - 0s 93ms/step
Test image pair 1 similarity: 0.48


IndexError: list index out of range